# Kusur Azaldıkça Kalite Artar

## Hazırlık

In [1]:
# gerekli kütüphaneler
import torch

# bunlar bize hep lazım
alfabe = list('.abcçdefgğhıijklmnoöprsştuüvyz')
harf2idx = { harf:idx for idx, harf in enumerate(alfabe) }
idx2harf = { idx:harf for harf, idx in harf2idx.items() }

# bize lazım olan işlem: bir string alıp başına ve sonuna nokta ekle
def isle(x):
  return ['.'] + list(x) + ['.']
# bunu da çok kullanacağız
def bigramGetir(x):
  return zip(x, x[1:])

# isimleri okurken işleyelim
isimler =  list(map(lambda satir: isle(satir), open("./isimler.txt", "r").read().splitlines()))
len(isimler), isimler[0]

(29996, ['.', 'a', 'b', 'a', 'c', 'a', '.'])

## Kaldığımız Yer

In [2]:
M = torch.ones((30, 30), dtype = torch.float)
for isim in isimler:
  for h1, h2 in bigramGetir(isim):
    M[harf2idx[h1], harf2idx[h2]] += 1

P = M / M.sum(dim = 1, keepdim = True)
P.sum()

tensor(30.0000)

- Harfleri eşit olasılıkla dizerek elimizdekilere benzer bir şey üretmemizin mümkün olmadığına kanaat getirdik.
- "İsimleri oluşturan harfler hangi olasılıkla göre birbirini takip ediyor?" sorusuna yukarıdaki kod ile cevap aradık.
- Olasılıklara dikkat ederek çöp elde etmesek de kaliteli bir çıktı üretemedik.

### Kalite Derken

"Ölçemezsek iyileştiremeyeceğimize" göre _kalite_ istiyorsak bunu kantitatif bir şekilde ifade edebilmeliyiz. Önce sözlü ifade edelim: modelin ürettiği yeni ismin, veri setinin istatistiksel yapısına uyumu.

Modelimiz "hiçbir şey öğrenmeseydi" her harf için 1/30 olasılığa takılı kalacaktık. Fakat aşağıdaki _abacılar_ örneğinde görüyoruz ki "eşit ihtimalin" ötesinde bir bilgi elde edebilmişiz. Bu isim için ne kadar bilgi öğrendiğimizi gösteren bir değer üretelim:

[MLE](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) yöntemi ile bu ismi oluşturan bigramların olasılıklarını çarparak özetleyelim. Elde ettiğimiz bu sayı 0 (en kötü) ve 1 (en iyi) arasında bir değer olacak.

Burada mini çakallıklar yapıp sadece daha okunaklı bir değere ulaşmak adına `MLE`'den `NLL`'ye terfi edeceğiz:

- `MLE` ile hesapladığımız değer 0-1 arasına sıkıştığı için [log](https://en.wikipedia.org/wiki/Likelihood_function#Log-likelihood)'unu alıp 0 ile -∞ arasına genişletiyoruz. Aşağıda göreceğiniz gibi değerler özünde aynı olsa da MLE gözünüzü kanatabiliyor.
- Bu (`LL`) değerin tersini alarak temiz bir ölçü elde ediyoruz (`Negative LL`: `NLL`).
- `NLL` değerini öğe adedi ile bölerek kullanırsak biraz daha normalleştirebiliriz, yine aynı şeyi ifade ediyor olsak da bu gösterim daha okunaklı.

`NLL` bizim için bir `loss function`: 0 ise mükemmel, ne kadar büyükse o kadar kötü. Birazdan buna odaklanacağız.

Bununla birlikte bir mini çakallık daha yaptık: matrisi `torch.zeroes` yerine `torch.ones` ile başlattık ki (mesela) _jj_ veri setinde olmasa da 1 kez görmüş gibi saydık ve `log(0)` sorunundan kurtulduk. Camia buna `smoothing` diyor.

Bakalım:

In [3]:
def prn(isim):
  mle = 1.0
  ll = 0.0
  n = 0
  print(f'------------------')
  print(f'---Öğeler:--------')
  print(f'------------------')
  for h1, h2 in bigramGetir(isle(isim)):
    olasilik = P[harf2idx[h1], harf2idx[h2]]
    adet = M[harf2idx[h1], harf2idx[h2]]
    topl = M[harf2idx[h1]].sum()
    mle *= olasilik
    logOl = torch.log(olasilik)
    ll += logOl
    n += 1
    print(f'{h1}{h2}: {olasilik:.4f} ({adet:05.0f} / {topl:05.0f})')
  
  print(f'------------------')
  print(f'---Özet:----------')
  print(f'------------------')
  print(f'MLE: {mle:.11f}')
  print(f'LL :      {ll:.4f}')
  print(f'NLL:        {-ll/n:.4f}')

prn('abacılar')

------------------
---Öğeler:--------
------------------
.a: 0.1046 (03141 / 30026)
ab: 0.0237 (00883 / 37311)
ba: 0.4033 (02514 / 06234)
ac: 0.0273 (01017 / 37311)
cı: 0.2646 (01236 / 04671)
ıl: 0.0808 (01089 / 13479)
la: 0.2073 (03822 / 18438)
ar: 0.1736 (06479 / 37311)
r.: 0.2460 (04201 / 17080)
------------------
---Özet:----------
------------------
MLE: 0.00000000515
LL :      -19.0841
NLL:        2.1205


Elimiz değmişken tüm model için `NLL` hesaplayalım; belki lazım olur:

In [4]:
ll = 0.0
n = 0
for isim in isimler:
  for h1, h2 in bigramGetir(isim):
    ll += torch.log(P[harf2idx[h1], harf2idx[h2]])
    n += 1

print(f'-----------')
print(f'---Tümü:---')
print(f'-----------')
print(f'NLL: {-ll/n:.4f}')

-----------
---Tümü:---
-----------
NLL: 2.5279


_abacılar_ isminin nasıl olası olduğunu gördük; olmayası bir örnek verelim:

In [5]:
prn('jöjö')

------------------
---Öğeler:--------
------------------
.j: 0.0000 (00001 / 30026)
jö: 0.0238 (00001 / 00042)
öj: 0.0003 (00001 / 03669)
jö: 0.0238 (00001 / 00042)
ö.: 0.0003 (00001 / 03669)
------------------
---Özet:----------
------------------
MLE: 0.00000000000
LL :      -34.2005
NLL:        6.8401


## Özetle

- Kusur için bir ölçü belirledik, bunu azalttığımızda kaliteyi artıracağımızı gördük.
- Bu ölçüye göre isimlerin ne kadar olası veya _olmayası_ olduğuna dair bir fikrimiz var artık.
- Akla gelen en makul yöntem ile biraz ilerleme kaydettik. Harfleri üçer, dörder gruplamak bir sonraki doğal adım olarak görünebilir ama bu yaklaşımı büyük ölçekte çalıştırmak zor: 3 harflik bir bağlam oluşturduk diyelim bir anda 27000x27000 bir matrisle çalışıyor olacağız ve adetler anlamsız hale gelecek kadar küçülecek.